In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from ipywidgets import widgets, interact
from IPython.display import display


In [2]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import re
from ipywidgets import interact, widgets

# Load your dataset (example path provided, adjust as needed)
data = pd.read_excel('Data Analysis Automation - Polyiso 2024.xlsx', sheet_name='Data set')

# Check if 'DESCRIPTION' column exists
if 'DESCRIPTION' not in data.columns:
    raise KeyError("The column 'DESCRIPTION' does not exist in the dataset.")

# Global variables to hold the updated and final DataFrames
updated_data = data.copy()
final_df_global = None
new_df = None  # Variable to hold the DataFrame after target column selection
granularity_choice = 'Upto Top and Bottom Facer'  # Default value, can be changed by user

# Function to convert columns to numeric with exceptions
def convert_columns_to_numeric(df, exceptions=[]):
    for col in df.columns:
        if col not in exceptions:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

# Specify the columns to exclude from conversion
exceptions = ['DESCRIPTION', 'SURFACTANT', 'SAP']

# Convert columns to numeric with exceptions
data = convert_columns_to_numeric(data, exceptions=exceptions)

# Initialize 'SAP-Desc' and 'Merge-Key' columns if they don't exist
if 'SAP-Desc' not in updated_data.columns:
    updated_data['SAP-Desc'] = updated_data['DESCRIPTION']
if 'Merge-Key' not in updated_data.columns:
    updated_data['Merge-Key'] = np.nan

def filter_dataframe(selected_values):
    # Filter the DataFrame based on the selected values
    filtered_df = updated_data[updated_data['DESCRIPTION'].isin(selected_values)]
    display(filtered_df)
    return filtered_df

def select_columns(filtered_df, selected_columns):
    final_df = filtered_df[list(selected_columns)]
    display(final_df)
    return final_df

def create_column_dropdown(filtered_df):
    column_options = filtered_df.columns.tolist()
    column_dropdown = widgets.SelectMultiple(
        options=column_options,
        value=[column_options[0]],
        description='Columns',
        disabled=False
    )
    return column_dropdown

def set_target_column(final_df, selected_column):
    global final_df_global
    
    # Make a copy of the final_df to avoid modifying the original DataFrame
    df_modified = final_df.copy()
    
    # Rename the selected column to 'GROUP'
    df_modified.rename(columns={selected_column: 'GROUP'}, inplace=True)
    
    # Move the 'GROUP' column to the last position
    cols = list(df_modified.columns)
    cols.append(cols.pop(cols.index('GROUP')))
    df_modified = df_modified[cols]
    
    # Convert all columns except 'SAP-Desc' or 'DESCRIPTION', 'GROUP', and 'Attributes' to numeric
    columns_to_exclude = ['SAP-Desc', 'DESCRIPTION', 'GROUP', 'Attributes']
    for col in df_modified.columns:
        if col not in columns_to_exclude:
            df_modified[col] = pd.to_numeric(df_modified[col], errors='coerce')
    
    # Update the global final_df_global
    final_df_global = df_modified

    # Display the modified DataFrame
    display(final_df_global)
    
    return final_df_global

def interactive_filter_and_select():
    global final_df_global
    result = {'filtered_df': None, 'final_df': None}

    def update_filtered_df(selected_values):
        filtered_df = filter_dataframe(selected_values)
        result['filtered_df'] = filtered_df

        # Create column selection dropdown after filtering
        column_dropdown = create_column_dropdown(filtered_df)
        display(column_dropdown)

        def update_final_df(selected_columns):
            final_df = select_columns(filtered_df, selected_columns)
            result['final_df'] = final_df
            final_df_global = final_df  # Store the final DataFrame in the global variable

            # Create the dropdown widget for selecting the target column
            column_options = final_df.columns.tolist()
            target_column_dropdown = widgets.Dropdown(
                options=column_options,
                value=column_options[0],
                description='Target Column',
                disabled=False
            )

            def interactive_update(selected_column):
                global final_df_global, new_df
                new_df = set_target_column(final_df, selected_column)
                # Save the new_df for the next steps in the second code block
                final_df_global = new_df

            print("Choose a target column: This column will be used to set up the groups for the analysis")
            interact(interactive_update, selected_column=target_column_dropdown)

        interact(update_final_df, selected_columns=column_dropdown)

    interact(update_filtered_df, selected_values=description_dropdown)
    return description_dropdown, result

# Create a dropdown widget for granularity selection
granularity_options = ['Upto Top and Bottom Facer', 'Upto Dimensions']
granularity_dropdown = widgets.Dropdown(
    options=granularity_options,
    value=granularity_options[0],
    description='Granularity',
    disabled=False
)

# Create a dropdown widget for DESCRIPTION selection
description_options = data['DESCRIPTION'].unique().tolist()
description_dropdown = widgets.SelectMultiple(
    options=description_options,
    value=[description_options[0]],
    description='Description',
    disabled=False
)

def handle_granularity_change(granularity_choice_value):
    global granularity_choice
    granularity_choice = granularity_choice_value

    if granularity_choice == 'Upto Top and Bottom Facer':
        # Proceed with the current workflow
        dropdown_widget, result = interactive_filter_and_select()
    else:
        print("Currently, only 'Upto Top and Bottom Facer' is implemented in this block.")

interact(handle_granularity_change, granularity_choice_value=granularity_dropdown)

interactive(children=(Dropdown(description='Granularity', options=('Upto Top and Bottom Facer', 'Upto Dimensio…

<function __main__.handle_granularity_change(granularity_choice_value)>

In [3]:
from IPython.display import display, HTML
from scipy import stats

outliers_df_global = pd.DataFrame(columns=['Attributes', 'Column', 'Outlier'])

def extract_attributes(description, granularity_choice):
    if granularity_choice == 'Upto Dimensions':
        parts = description.split(" ")
        attributes = " ".join(parts[:-2]) + "-" + parts[-2] + " " + parts[-1]
        return attributes
    else:
        match = re.match(r"(\d+\.\d+|\d+|\b[a-zA-Z]\b)\s+(.+?)\s+\d", description)
        if match:
            thickness = match.group(1)
            material_type = match.group(2)
            return f"{thickness} {material_type}"
        return None

def grubbs_test(data, alpha=0.05):
    n = len(data)
    if n < 3:  # Grubbs' test requires at least 3 data points
        return None, None
    mean_y = np.mean(data)
    std_y = np.std(data, ddof=1)
    if std_y == 0:
        return None, None
    numerator = np.max(np.abs(data - mean_y))
    grubbs_statistic = numerator / std_y
    
    t_dist = stats.t.ppf(1 - alpha / (2 * n), n - 2)
    critical_value = ((n - 1) / np.sqrt(n)) * np.sqrt(t_dist ** 2 / (n - 2 + t_dist ** 2))
    
    return grubbs_statistic, critical_value

def detect_outliers(final_df, granularity_choice, alpha=0.05):
    global outliers_df_global

    # Determine the appropriate column to use based on granularity choice
    if granularity_choice == 'Upto Top and Bottom Facer':
        column_to_use = 'DESCRIPTION'
    else:
        column_to_use = 'SAP-Desc'

    # Create a copy of the DataFrame to avoid SettingWithCopyWarning
    final_df = final_df.copy()

    final_df['Attributes'] = final_df[column_to_use].apply(lambda x: extract_attributes(x, granularity_choice))

    # List to capture outliers
    outliers_list = []

    grouped = final_df.groupby('Attributes')
    for attr, group_df in grouped:
        for column in group_df.columns:
            if group_df[column].dtype != object and column != 'GROUP':  # Ensure the column is numeric
                data_column = pd.to_numeric(group_df[column], errors='coerce').dropna().values
                while True:
                    grubbs_statistic, critical_value = grubbs_test(data_column, alpha)
                    if grubbs_statistic is None or critical_value is None:
                        break
                    if grubbs_statistic > critical_value:
                        outlier = data_column[np.argmax(np.abs(data_column - np.mean(data_column)))]
                        outliers_list.append({'Attributes': attr, 'Column': column, 'Outlier': outlier})
                        data_column = data_column[data_column != outlier]
                    else:
                        break

    outliers_df_global = pd.DataFrame(outliers_list)

    # Display the detected outliers DataFrame
    display(HTML("<b>Detected Outliers DataFrame:</b>"))
    display(outliers_df_global)

def add_attributes_column(final_df, granularity_choice):
    if granularity_choice == 'Upto Top and Bottom Facer':
        final_df['Attributes'] = final_df['DESCRIPTION'].apply(lambda x: extract_attributes(x, granularity_choice))
    else:
        final_df['Attributes'] = final_df['SAP-Desc'].apply(lambda x: extract_attributes(x, granularity_choice))
    return final_df

def prompt_outlier_removal(final_df, outliers_df, granularity_choice):
    # Add the 'Attributes' column to the DataFrame
    final_df = add_attributes_column(final_df, granularity_choice)

    # Display the detected outliers DataFrame
    display(HTML("<b>Detected Outliers DataFrame:</b>"))
    display(outliers_df)
    
    # If no outliers are detected, print a message and return
    if outliers_df.empty:
        print("No outliers detected.")
        return final_df
    
    # Iterate over each outlier and ask the user whether to remove it
    for index, row in outliers_df.iterrows():
        attr = row['Attributes']
        column = row['Column']
        outlier_value = row['Outlier']
        
        # Ask the user for a decision
        decision = input(f"Do you want to remove the outlier {outlier_value} from {attr} in column {column}? (Yes/No): ").strip().lower()
        
        if decision == 'yes':
            final_df = final_df[~((final_df[column] == outlier_value) & (final_df['Attributes'] == attr))]
            print(f"Outlier {outlier_value} from {attr} in column {column} removed.")
        else:
            print(f"Outlier {outlier_value} from {attr} in column {column} not removed.")
    
    return final_df

def calculate_statistics(df, description_column, group_column, granularity_choice):
    df['Attributes'] = df[description_column].apply(lambda x: extract_attributes(x, granularity_choice))
    
    # Drop rows where attributes could not be extracted
    df = df.dropna(subset=['Attributes'])
    
    # Group by the extracted attributes and the specified group column
    grouped = df.groupby(['Attributes', group_column])
    
    def group_stats(group):
        group = group.drop(columns=[description_column, 'Attributes', group_column])
        stats = {
            'N': group.shape[0],
            'N*': group.isnull().sum(),
            'Mean': group.mean(numeric_only=True),
            'StDev': group.std(numeric_only=True),
            'Minimum': group.min(numeric_only=True),
            'Median': group.median(numeric_only=True),
            'Maximum': group.max(numeric_only=True),
            'Skewness': group.skew(numeric_only=True)
        }
        return pd.DataFrame(stats)
    
    stats_df = grouped.apply(group_stats).reset_index()
    return stats_df

def calculate_and_display_statistics(final_df, granularity_choice):
    global table1_df_global

    if granularity_choice == 'Upto Top and Bottom Facer':
        description_column = 'DESCRIPTION'
        group_column = 'GROUP'
    else:
        description_column = 'SAP-Desc'
        group_column = 'GROUP'
        
    stats_df = calculate_statistics(final_df, description_column, group_column, granularity_choice)
    stats_df.rename(columns={'level_2': 'Variable'}, inplace=True)

    columns_order = ['Attributes', 'GROUP', 'Variable', 'N', 'N*', 'Mean', 'StDev', 'Minimum', 'Median', 'Maximum', 'Skewness']
    stats_df = stats_df[columns_order].round(3)

    pd.options.display.float_format = '{:.6f}'.format

    for col in ['Mean', 'StDev', 'Minimum', 'Median', 'Maximum', 'Skewness']:
        if col in stats_df.columns:
            stats_df[col] = stats_df[col].apply(lambda x: f"{x:.3f}" if pd.notnull(x) else x)

    table1_df = stats_df.sort_values(by=['Attributes', 'Variable', 'GROUP']).reset_index().drop(columns=['index'])
    table1_df_global = table1_df

    display(HTML("<b>Statistics DataFrame:</b>"))
    display(table1_df)

# Ensure final_df_global is assigned properly in the first block
if final_df_global is not None:
    final_df_global = add_attributes_column(final_df_global, granularity_choice)
    detect_outliers(final_df_global, granularity_choice, alpha=0.05)
    final_df_global = prompt_outlier_removal(final_df_global, outliers_df_global, granularity_choice)
    calculate_and_display_statistics(final_df_global, granularity_choice)
else:
    print("final_df_global is not yet defined. Please complete the interactive selections in the first code block.")


,Attributes,Column,Outlier
0,0.5 High Density Cgf-Cgf,Comps,123.21
1,0.5 High Density Cgf-Cgf,Comps,90.35
2,2.6 Flat Blk-Blk,EdgeCol_Top,0.26
3,2.6 Flat Blk-Blk,W_Shrink,0.00


,Attributes,Column,Outlier
0,0.5 High Density Cgf-Cgf,Comps,123.21
1,0.5 High Density Cgf-Cgf,Comps,90.35
2,2.6 Flat Blk-Blk,EdgeCol_Top,0.26
3,2.6 Flat Blk-Blk,W_Shrink,0.00


Do you want to remove the outlier 123.21 from 0.5 High Density Cgf-Cgf in column Comps? (Yes/No):  no


Outlier 123.21 from 0.5 High Density Cgf-Cgf in column Comps not removed.


Do you want to remove the outlier 90.35 from 0.5 High Density Cgf-Cgf in column Comps? (Yes/No):  no


...

Do you want to remove the outlier 0.26 from 2.6 Flat Blk-Blk in column EdgeCol_Top? (Yes/No):  no


Outlier 0.26 from 2.6 Flat Blk-Blk in column EdgeCol_Top not removed.


Do you want to remove the outlier 0.0 from 2.6 Flat Blk-Blk in column W_Shrink? (Yes/No):  no


Outlier 0.0 from 2.6 Flat Blk-Blk in column W_Shrink not removed.


,Attributes,GROUP,Variable,N,N*,Mean,StDev,Minimum,Median,Maximum,Skewness
0,0.5 High Density Cgf-Cgf,EVONIK,Comps,64,0,107.776,4.468,98.580,107.380,123.210,0.679
1,0.5 High Density Cgf-Cgf,SILSTAB,Comps,38,0,103.087,3.754,90.350,103.315,110.640,-0.915
2,0.5 High Density Cgf-Cgf,EVONIK,Comps_Thick,64,0,104.908,6.349,90.600,103.770,122.070,0.414
3,0.5 High Density Cgf-Cgf,SILSTAB,Comps_Thick,38,0,101.942,7.876,81.960,102.790,115.890,-0.450
4,0.5 High Density Cgf-Cgf,EVONIK,Comps_Thin,64,0,107.266,7.881,88.610,107.435,125.840,-0.098
...,...,...,...,...,...,...,...,...,...,...,...
75,Q Taper Blk-Blk,SILSTAB,Kfactor_Init-1,18,0,0.000,0.000,0.000,0.000,0.000,0.000
76,Q Taper Blk-Blk,EVONIK,L_Shrink,23,0,0.000,0.000,0.000,0.000,0.000,0.000
77,Q Taper Blk-Blk,SILSTAB,L_Shrink,18,0,0.000,0.000,0.000,0.000,0.000,0.000
78,Q Taper Blk-Blk,EVONIK,W_Shrink,23,0,0.000,0.000,0.000,0.000,0.000,0.000


In [4]:
filtered_data=final_df_global

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import levene, f_oneway, kruskal, median_test
from statsmodels.stats.weightstats import CompareMeans, DescrStatsW
from fuzzywuzzy import fuzz
from ipywidgets import interact, widgets
from IPython.display import display, HTML
import warnings
import io
from PIL import Image
warnings.filterwarnings('ignore')

# Global variable to store results
global_results = []
global_levene_results = []
global_anova_welch_results = []
user_choice_2 = None  # Initialize user_choice_2
plots_dict = {}  # Dictionary to store plots

# Assuming filtered_data and table1_df are already loaded

def anderson_darling_test(data):
    data = np.asarray(data).flatten()
    result = stats.anderson(data, dist='norm')
    return result.statistic, result.critical_values, result.significance_level

# Function to find representative description
def find_representative_description(descriptions, threshold=80):
    unique_descriptions = np.unique(descriptions)
    representative_description = unique_descriptions[0]
    for desc in unique_descriptions[1:]:
        if fuzz.ratio(representative_description, desc) >= threshold:
            continue
        else:
            representative_description = desc
            break
    return representative_description

# Function to plot standard deviations
def plot_standard_deviations(column, group_labels, means, std_devs, p_value_1, p_value_2):
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.errorbar(means, group_labels, xerr=std_devs, fmt='o', capsize=5, capthick=2, elinewidth=2, linestyle='None')
    ax.set_title(f'Test for Equal Variances: {column} vs GROUP\nMultiple comparison intervals for the standard deviation, α = 0.05')
    ax.set_ylabel('GROUP')
    ax.set_xlabel('Standard Deviation')
    plt.figtext(0.95, 0.9, f"Multiple Comparisons\nP-Value (Levene's Test)\nP-Value: {p_value_1:.3f}", bbox={"facecolor": "lightgray", "alpha": 0.5, "pad": 5})
    plt.figtext(0.95, 0.8, f"P-Value (ANOVA or Welch)\nP-Value: {p_value_2:.3f}", bbox={"facecolor": "lightgray", "alpha": 0.5, "pad": 5})
    ax.set_yticks(range(len(group_labels)))
    ax.set_yticklabels(group_labels)
    plt.close(fig)
    return fig

# Function to handle user's choice for EVT or NPT
def handle_choice_1(choice):
    global user_choice_1
    user_choice_1 = choice
    print(f"User choice: {user_choice_1}")
    if user_choice_1 == 'Equal Variance Test':
        equal_variance_test()
    elif user_choice_1 == 'Non-Parametric Test':
        check_extreme_outliers()

# Function to handle user's choice for non-parametric test
def handle_choice_2(attr, group_df):
    def handle_choice_test(choice):
        global user_choice_2, global_results
        user_choice_2 = choice
        print(f"User choice for {attr}: {user_choice_2}")

        # Remove existing results for the chosen attribute and test type
        global_results = [result for result in global_results if not (result['Attributes'] == attr and result['Test Conducted'] == user_choice_2)]
        
        if user_choice_2 == "Mood's Median":
            results = moods_median_test(group_df, attr)
        elif user_choice_2 == 'Kruskal-Wallis':
            results = kruskal_wallis_test(group_df, attr)
        
        global_results.extend(results)
        results_df = pd.DataFrame(results)
        display(results_df)
        return results

    interact_widget = interact(handle_choice_test, choice=widgets.Dropdown(options=["Mood's Median", 'Kruskal-Wallis'], description='Choose Test:'))
    return interact_widget.widget.result

# Function to check skewness and proceed
def check_skewness_and_proceed():
    table1_df_global['Skewness'] = pd.to_numeric(table1_df_global['Skewness'], errors='coerce')
    skewness_check = table1_df_global['Skewness'].apply(lambda x: -2 <= x <= 2 if pd.notna(x) else True)
    if skewness_check.all():
        print("Skewness for all records is between -2 and 2. Proceed to user choice.")
        interact(handle_choice_1, choice=widgets.Dropdown(options=['Equal Variance Test', 'Non-Parametric Test'], description='Choose Test:'))
    else:
        print("Skewness for some records is outside the range of -2 and 2. Suggesting Non-Parametric Test (NPT) instead.")
        interact(handle_choice_1, choice=widgets.Dropdown(options=['Equal Variance Test', 'Non-Parametric Test'], description='Choose Test:'))

# Perform Anderson-Darling Test
def perform_anderson_darling_test():
    numerical_columns = [col for col in filtered_data.columns if pd.api.types.is_numeric_dtype(filtered_data[col])]
    print(f"Numerical columns: {numerical_columns}")
    data_for_testing = filtered_data[numerical_columns].values.flatten()
    statistic, critical_values, significance_level = anderson_darling_test(data_for_testing)
    p_value_threshold = 0.05
    if statistic < critical_values[2]:
        result = "Yes, fail to reject null"
        print(f"AD Test Result = {result}")
        print("p>0.05  fail to reject null")
        check_skewness_and_proceed()
    else:
        result = "No, reject null"
        print(f"AD Test Result = {result}")
        print("p<=0.05  reject null")
        perform_mgrt_test()

# Function to perform Minimum Group Records Test (MGRT)
def perform_mgrt_test():
    group_counts = filtered_data.groupby(['Attributes', 'GROUP']).size()
    print("Group counts:\n", group_counts)
    num_unique_groups = group_counts.groupby(level=0).size()
    print(f"Number of unique groups: {num_unique_groups}")
    result_MGRT = "Yes"
    for attr, group_count in group_counts.groupby(level=0):
        if 2 <= len(group_count) <= 9:
            print(f"Number of unique groups for {attr} is between 2 and 9.")
            if not all(group_count > 15):
                result_MGRT = "No"
                print(f"At least one group in {attr} has 15 or fewer records.")
            else:
                print(f"All groups in {attr} have more than 15 records.")
        elif 10 <= len(group_count) <= 12:
            print(f"Number of unique groups for {attr} is between 10 and 12.")
            if not all(group_count > 20):
                result_MGRT = "No"
                print(f"At least one group in {attr} has 20 or fewer records.")
            else:
                print(f"All groups in {attr} have more than 20 records.")
        else:
            print(f"Number of unique groups for {attr} is outside the specified range (2-12).")
    print(f"Result from Minimum Group Records Test = {result_MGRT}")
    if result_MGRT == "Yes":
        check_skewness_and_proceed()
    else:
        interact(handle_choice_1, choice=widgets.Dropdown(options=['Equal Variance Test', 'Non-Parametric Test'], description='Choose Test:'))

# Function to perform Equal Variance Test
def equal_variance_test():
    global global_levene_results, global_anova_welch_results
    descriptions = filtered_data['Attributes']
    representative_description = find_representative_description(descriptions)
    levene_results = []
    anova_welch_results = []
    numeric_columns = filtered_data.select_dtypes(include='number').columns
    grouped = filtered_data.groupby('Attributes')
    for attr, group_df in grouped:
        print(f"Processing group: Attributes={attr}")
        for column in numeric_columns:
            if column != 'GROUP':
                groups = group_df.groupby('GROUP')[column].apply(lambda x: x.dropna().values)
                group_labels = [label for label, group in zip(groups.index, groups) if len(group) > 0]
                means = [np.mean(group) for group in groups if len(group) > 0]
                std_devs = [np.std(group) for group in groups if len(group) > 0]
                if not means or not std_devs:
                    continue
                print(f"\nTesting column: {column}")
                for label, group in zip(group_labels, groups):
                    print(f"Group: {label}, Data: {group}")
                try:
                    stat, p_value_1 = levene(*[group for group in groups if len(group) > 0])
                    if np.isnan(p_value_1):
                        raise ValueError("NaN P-Value")
                except:
                    p_value_1 = np.nan
                    levene_results.append({
                        'Column': column,
                        'Levene P-Value': 'N/A',
                        'Result': 'N/A',
                        'Attributes': attr
                    })
                    anova_welch_results.append({
                        'Column': column,
                        'ANOVA/Welch P-Value': 'N/A',
                        'Result': 'N/A',
                        'Test Type': 'N/A',
                        'Attributes': attr
                    })
                    continue
                levene_result = 'Equal' if p_value_1 > 0.05 else 'Diff'
                levene_results.append({
                    'Column': column,
                    'Levene P-Value': p_value_1,
                    'Levene Statistic': stat,
                    'Result': levene_result,
                    'Attributes': attr
                })
                try:
                    if p_value_1 > 0.05:
                        print(f"Yes, Parametric ANOVA for {column}")
                        anova_result = f_oneway(*[group for group in groups if len(group) > 0])
                        p_value_2 = anova_result.pvalue
                        if np.isnan(p_value_2):
                            raise ValueError("NaN P-Value")
                        if p_value_2 > 0.05:
                            result = 'Equal'
                            test_type = 'ANOVA'
                        else:
                            result = 'Diff'
                            test_type = 'ANOVA'
                    else:
                        print(f"No, Parametric Welch ANOVA for {column}")
                        desc_stats = [DescrStatsW(group) for group in groups if len(group) > 0]
                        cm = CompareMeans(*desc_stats)
                        welch_result = cm.ttest_ind(usevar='unequal')
                        p_value_2 = welch_result[1]
                        if np.isnan(p_value_2):
                            raise ValueError("NaN P-Value")
                        if p_value_2 > 0.05:
                            result = 'Equal'
                            test_type = 'Welch ANOVA'
                        else:
                            result = 'Diff'
                            test_type = 'Welch ANOVA'
                except:
                    p_value_2 = np.nan
                    result = 'N/A'
                    test_type = 'N/A'
                fig = plot_standard_deviations(column, group_labels, means, std_devs, p_value_1, p_value_2)
                plots_dict.setdefault('standard_deviations', []).append((f'{attr}_{column}', fig))
                anova_welch_results.append({
                    'Column': column,
                    'ANOVA/Welch P-Value': p_value_2,
                    'Result': result,
                    'Test Type': test_type,
                    'Attributes': attr
                })
    global_levene_results = pd.DataFrame(levene_results)
    global_anova_welch_results = pd.DataFrame(anova_welch_results)
    # Format the DataFrames as requested
    levene_df_pivot = pd.pivot_table(global_levene_results, index='Attributes', columns='Column', values='Levene P-Value', aggfunc='first')
    anova_welch_df_pivot = pd.pivot_table(global_anova_welch_results, index='Attributes', columns='Column', values='ANOVA/Welch P-Value', aggfunc='first')
    # DataFrames without p-values
    levene_df_simple = global_levene_results.pivot(index='Attributes', columns='Column', values='Result')
    anova_welch_df_simple = global_anova_welch_results.pivot(index='Attributes', columns='Column', values='Result')
    print("Levene's Test Results (with P-Values):")
    display(levene_df_pivot)
    print("\nANOVA/Welch Test Results (with P-Values):")
    display(anova_welch_df_pivot)
    print("\nLevene's Test Results (Simplified):")
    display(levene_df_simple)
    print("\nANOVA/Welch Test Results (Simplified):")
    display(anova_welch_df_simple)

# Function to perform Mood's Median Test
def moods_median_test(group_df, attr):
    numeric_columns = group_df.select_dtypes(include='number').columns
    results = []
    for column in numeric_columns:
        if column != 'GROUP':
            groups = [group[column].dropna().values for name, group in group_df.groupby('GROUP')]
            groups = [group for group in groups if len(group) > 0]
            if len(groups) < 2:
                continue
            try:
                stat, p_value, med, tbl = median_test(*groups)
                if p_value > 0.05:
                    result = 'Equal'
                else:
                    result = 'Diff'
                group_stats = group_df.groupby('GROUP')[column].describe().unstack()
                result_data = {
                    'Attributes': attr,
                    'Column': column,
                    'Group': group_df['GROUP'].unique().tolist(),
                    'Median': group_stats['50%'].to_dict(),
                    'N': group_stats['count'].to_dict(),
                    'Q1-Q3': group_stats[['25%', '75%']].to_dict(),
                    'H-Value': stat,
                    'P-Value': p_value,
                    'Result': result,
                    'Test Conducted': "Mood's Median"
                }
                results.append(result_data)
            except ValueError as e:
                result_data = {
                    'Attributes': attr,
                    'Column': column,
                    'Group': group_df['GROUP'].unique().tolist(),
                    'Median': np.nan,
                    'N': len(group_df),
                    'Q1-Q3': 'N/A',
                    'H-Value': np.nan,
                    'P-Value': np.nan,
                    'Result': 'N/A',
                    'Test Conducted': "Mood's Median"
                }
                results.append(result_data)
    return results

# Function to perform Kruskal-Wallis Test
def kruskal_wallis_test(group_df, attr):
    numeric_columns = group_df.select_dtypes(include='number').columns
    results = []
    for column in numeric_columns:
        if column != 'GROUP':
            groups = [group[column].dropna().values for name, group in group_df.groupby('GROUP')]
            groups = [group for group in groups if len(group) > 0]
            if len(groups) < 2:
                continue
            try:
                stat, p_value = kruskal(*groups)
                if p_value > 0.05:
                    result = 'Equal'
                else:
                    result = 'Diff'
                group_stats = group_df.groupby('GROUP')[column].describe().unstack()
                result_data = {
                    'Attributes': attr,
                    'Column': column,
                    'Group': group_df['GROUP'].unique().tolist(),
                    'Median': group_stats['50%'].to_dict(),
                    'N': group_stats['count'].to_dict(),
                    'Q1-Q3': group_stats[['25%', '75%']].to_dict(),
                    'H-Value': stat,
                    'P-Value': p_value,
                    'Result': result,
                    'Test Conducted': "Kruskal-Wallis"
                }
                results.append(result_data)
            except ValueError as e:
                result_data = {
                    'Attributes': attr,
                    'Column': column,
                    'Group': group_df['GROUP'].unique().tolist(),
                    'Median': np.nan,
                    'N': len(group_df),
                    'Q1-Q3': 'N/A',
                    'H-Value': np.nan,
                    'P-Value': np.nan,
                    'Result': 'N/A',
                    'Test Conducted': "Kruskal-Wallis"
                }
                results.append(result_data)
    return results

# Function to perform tests by attribute
def perform_tests_by_attribute():
    all_results = []
    for attr in filtered_data['Attributes'].unique():
        group_df = filtered_data[filtered_data['Attributes'] == attr]
        print(f"\nChoose the test to conduct for {attr}:")
        results_df = handle_choice_2(attr, group_df)
        if results_df is not None:  # Ensure the results are valid
            all_results.append(results_df)

    # Collect the results from each attribute
    all_results_list = []
    for results_df in all_results:
        if isinstance(results_df, list):  # Check if it's a list
            all_results_list.extend(results_df)

    # Update the global results list
    global global_results
    global_results.extend(all_results_list)

# Function to check extreme outliers
def check_extreme_outliers():
    def find_extreme_outliers(series):
        q1 = series.quantile(0.25)
        q3 = series.quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        mean = series.mean()
        outliers = series[(series < lower_bound) | (series > upper_bound)]
        distances = (outliers - mean).abs()
        return outliers, distances

    outlier_present = False
    outliers_list = []
    numeric_columns = filtered_data.select_dtypes(include='number').columns
    grouped = filtered_data.groupby('Attributes')
    for attr, group_df in grouped:
        for column in numeric_columns:
            if column != 'GROUP':
                valid_groups = group_df['GROUP'].dropna().unique()
                if len(valid_groups) > 0:  # Ensure there are valid groups to plot
                    groups = group_df.groupby('GROUP')[column].apply(lambda x: x.dropna().values)
                    valid_groups = [group for group in groups if len(group) > 0]
                    if len(valid_groups) < 2:
                        continue
                    plt.figure(figsize=(10, 6))
                    sns.boxplot(x='GROUP', y=column, data=group_df)
                    plt.title(f"Boxplot of {column} by GROUP for Attributes={attr}")
                    plt.close()
                    plots_dict.setdefault('boxplot', []).append((f'{attr}_{column}', plt.gcf()))
                for name, group in group_df.groupby('GROUP'):
                    outliers, distances = find_extreme_outliers(group[column])
                    if not outliers.empty:
                        outlier_present = True
                        print(f"Column: {column}, Group: {name}, Attributes: {attr}")
                        print("Outliers and their distances from the mean:")
                        for value, distance in zip(outliers, distances):
                            print(f"Outlier: {value}, Distance from mean: {distance}")
                            outliers_list.append({
                                'Column': column,
                                'Group': name,
                                'Attributes': attr,
                                'Outlier': value,
                                'Distance from Mean': distance
                            })
    result = "Yes" if outlier_present else "No"
    print(f"Extreme outliers present: {result}")
    outliers_df = pd.DataFrame(outliers_list)
    display(outliers_df)
    perform_tests_by_attribute()

# Plotting functions for visualization
def plot_tukey_CIs(data, group_col, value_col):
    try:
        from statsmodels.stats.multicomp import pairwise_tukeyhsd
        tukey_result = pairwise_tukeyhsd(endog=data[value_col], groups=data[group_col], alpha=0.05)
        tukey_result.plot_simultaneous()
        plt.title(f'Tukey Simultaneous 95% CIs\nDifferences of Means for {value_col}')
        plt.xlabel('Difference of Means')
        plt.grid()
        plt.close()
        return plt.gcf()
    except Exception as e:
        print(f"Skipping Tukey HSD plot for {value_col}: {e}")
        return None

import seaborn as sns
import matplotlib.pyplot as plt

def plot_interval(data, group_col, value_col):
    # Plot points without the join parameter
    sns.pointplot(x=group_col, y=value_col, data=data, capsize=.1, errwidth=1.5, errorbar='sd')
    
    # Add a line plot to connect the points
    plt.plot(data[group_col], data[value_col], linestyle='-', color='red')
    
    plt.title(f'Interval Plot of {value_col} vs {group_col}\n95% CI for the Mean')
    plt.ylabel(value_col)
    plt.xlabel(group_col)
    plt.grid()
    plt.close()
    return plt.gcf()


def plot_individual_value(data, group_col, value_col):
    # Plot individual points using stripplot with jitter for better visualization
    sns.stripplot(x=group_col, y=value_col, data=data, jitter=True)
    
    # Plot point estimates and error bars
    sns.pointplot(x=group_col, y=value_col, data=data, errorbar='sd', color='red')
    
    # Manually add lines if you want to connect the points
    plt.plot(data[group_col], data.groupby(group_col)[value_col].mean(), linestyle='-', color='red')

    # Set plot labels and title
    plt.title(f'Individual Value Plot of {value_col} vs {group_col}')
    plt.ylabel(value_col)
    plt.xlabel(group_col)
    
    # Add a grid for better readability
    plt.grid()
    
    # Close the plot to avoid displaying in non-interactive environments
    plt.close()
    
    # Return the current figure object
    return plt.gcf()

def plot_boxplot(data, group_col, value_col):
    # Drop rows with NaN values in the group or value columns
    data = data.dropna(subset=[group_col, value_col])
    
    # Check if there are at least two groups with data
    if data[group_col].nunique() < 2:
        print(f"Not enough groups to plot boxplot for {value_col}. Skipping.")
        return None
    
    try:
        # Plot the boxplot
        sns.boxplot(x=group_col, y=value_col, data=data)
        
        # Plot point estimates and error bars on top of the boxplot
        sns.pointplot(x=group_col, y=value_col, data=data, errorbar='sd', color='red')
        
        # Manually add lines if you want to connect the points
        plt.plot(data[group_col], data.groupby(group_col)[value_col].mean(), linestyle='-', color='red')

        # Set plot labels and title
        plt.title(f'Boxplot of {value_col}')
        plt.ylabel(value_col)
        plt.xlabel(group_col)
        
        # Add a grid for better readability
        plt.grid()
        
        # Close the plot to avoid displaying in non-interactive environments
        plt.close()
        
        # Return the current figure object
        return plt.gcf()
    except ValueError as e:
        print(f"Error while plotting boxplot for {value_col}: {e}")
        return None
        
def plot_residuals(data, group_col, value_col):
    from statsmodels.formula.api import ols

    # Sanitize column names for use in the formula
    sanitized_value_col = value_col.replace('-', '_').replace(' ', '_')
    sanitized_group_col = group_col.replace('-', '_').replace(' ', '_')
    
    # Drop rows with NaN or infinite values in the relevant columns
    data = data.dropna(subset=[value_col, group_col])
    data = data[np.isfinite(data[value_col])]
    
    # Ensure there is more than one unique group and enough data to perform OLS
    if data[sanitized_group_col].nunique() < 2 or len(data) < 2:
        print(f"Not enough data to fit OLS model for {value_col}. Skipping.")
        return None
    
    # Rename columns for OLS model compatibility
    data = data.rename(columns={value_col: sanitized_value_col, group_col: sanitized_group_col})
    
    # Fit the OLS model
    formula = f'{sanitized_value_col} ~ {sanitized_group_col}'
    try:
        model = ols(formula, data=data).fit()
    except ValueError as e:
        print(f"Error fitting OLS model for {value_col}: {e}")
        return None
    
    residuals = model.resid
    fitted = model.fittedvalues
    
    fig, axs = plt.subplots(2, 2, figsize=(12, 10))
    
    sns.histplot(residuals, bins=15, kde=True, ax=axs[0, 0])
    axs[0, 0].set_title('Histogram of Residuals')
    
    stats.probplot(residuals, dist="norm", plot=axs[0, 1])
    axs[0, 1].set_title('Normal Probability Plot')
    
    sns.scatterplot(x=fitted, y=residuals, ax=axs[1, 0])
    axs[1, 0].axhline(0, color='r', linestyle='--')
    axs[1, 0].set_title('Residuals vs Fitted Values')
    
    sns.lineplot(x=np.arange(len(residuals)), y=residuals, ax=axs[1, 1])
    axs[1, 1].set_title('Residuals vs Order')
    axs[1, 1].axhline(0, color='r', linestyle='--')
    
    plt.tight_layout()
    plt.close(fig)
    return fig

# Plot all visualizations for each numerical column against 'GROUP'
def plot_all_visualizations(data):
    group_col = 'GROUP'
    numeric_columns = data.select_dtypes(include='number').columns
    for value_col in numeric_columns:
        if value_col != group_col:
            print(f"Plotting for column: {value_col}")
            fig = plot_tukey_CIs(data, group_col, value_col)
            if fig:
                plots_dict.setdefault('tukey_CIs', []).append((value_col, fig))
            fig = plot_interval(data, group_col, value_col)
            plots_dict.setdefault('interval', []).append((value_col, fig))
            fig = plot_individual_value(data, group_col, value_col)
            plots_dict.setdefault('individual_value', []).append((value_col, fig))
            fig = plot_boxplot(data, group_col, value_col)
            plots_dict.setdefault('boxplot', []).append((value_col, fig))
            fig = plot_residuals(data, group_col, value_col)
            plots_dict.setdefault('residuals', []).append((value_col, fig))

# Save plots to Excel file
def save_plots_to_excel(plots_dict, filename):
    with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
        for plot_type, plots in plots_dict.items():
            worksheet_name = plot_type[:31]  # Worksheet names must be <= 31 chars
            worksheet = writer.book.add_worksheet(worksheet_name)
            row = 0
            for value_col, fig in plots:
                # Save the figure to a BytesIO object
                img_data = io.BytesIO()
                fig.savefig(img_data, format='png')
                img_data.seek(0)
                # Open the image with PIL
                image = Image.open(img_data)
                # Convert to a format xlsxwriter can handle
                image_data = io.BytesIO()
                image.save(image_data, format='png')
                image_data.seek(0)
                # Insert the image into the Excel sheet
                worksheet.insert_image(row, 0, '', {'image_data': image_data})
                worksheet.write(row, 1, value_col)
                row += 30  # Adjust the row position for the next plot

# Interact to trigger the Anderson-Darling test and proceed based on the results
perform_anderson_darling_test()

# Example usage with filtered_data
plot_all_visualizations(filtered_data)



Numerical columns: ['Core', 'Comps_Thin', 'Comps_Thick', 'Comps', 'F_Thick', 'Kfactor_Init-1', 'EdgeCol_Bot', 'EdgeCol_Top', 'L_Shrink', 'W_Shrink']
AD Test Result = No, reject null
p<=0.05  reject null
Group counts:
 Attributes                GROUP  
0.5 High Density Cgf-Cgf  EVONIK     64
                          SILSTAB    38
1.5 Flat Blk-Blk          EVONIK     36
                          SILSTAB    25
2.6 Flat Blk-Blk          EVONIK     15
                          SILSTAB    11
Q Taper Blk-Blk           EVONIK     23
                          SILSTAB    18
dtype: int64
Number of unique groups: Attributes
0.5 High Density Cgf-Cgf    2
1.5 Flat Blk-Blk            2
2.6 Flat Blk-Blk            2
Q Taper Blk-Blk             2
dtype: int64
Number of unique groups for 0.5 High Density Cgf-Cgf is between 2 and 9.
All groups in 0.5 High Density Cgf-Cgf have more than 15 records.
Number of unique groups for 1.5 Flat Blk-Blk is between 2 and 9.
All groups in 1.5 Flat Blk-Blk have more t

C:\Users\chilukalo\AppData\Local\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


interactive(children=(Dropdown(description='Choose Test:', options=('Equal Variance Test', 'Non-Parametric Tes…

Plotting for column: Core
Skipping Tukey HSD plot for Core: Axis limits cannot be NaN or Inf
Plotting for column: Comps_Thin
Plotting for column: Comps_Thick
Plotting for column: Comps
Plotting for column: F_Thick
Plotting for column: Kfactor_Init-1
Skipping Tukey HSD plot for Kfactor_Init-1: Axis limits cannot be NaN or Inf
Plotting for column: EdgeCol_Bot
Skipping Tukey HSD plot for EdgeCol_Bot: Axis limits cannot be NaN or Inf
Plotting for column: EdgeCol_Top
Skipping Tukey HSD plot for EdgeCol_Top: Axis limits cannot be NaN or Inf
Plotting for column: L_Shrink
Skipping Tukey HSD plot for L_Shrink: Axis limits cannot be NaN or Inf
Plotting for column: W_Shrink
Skipping Tukey HSD plot for W_Shrink: Axis limits cannot be NaN or Inf


<Figure size 640x480 with 0 Axes>

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import levene, f_oneway, kruskal, median_test
from statsmodels.stats.weightstats import CompareMeans, DescrStatsW
from fuzzywuzzy import fuzz
from ipywidgets import interact, widgets
from IPython.display import display, HTML
import warnings
import io
from PIL import Image
warnings.filterwarnings('ignore')

# Global variable to store results
global_results = []
global_levene_results = []
global_anova_welch_results = []
user_choice_2 = None  # Initialize user_choice_2
plots_dict = {}  # Dictionary to store plots

# Assuming filtered_data and table1_df are already loaded

def anderson_darling_test(data):
    data = np.asarray(data).flatten()
    result = stats.anderson(data, dist='norm')
    return result.statistic, result.critical_values, result.significance_level

# Function to find representative description
def find_representative_description(descriptions, threshold=80):
    unique_descriptions = np.unique(descriptions)
    representative_description = unique_descriptions[0]
    for desc in unique_descriptions[1:]:
        if fuzz.ratio(representative_description, desc) >= threshold:
            continue
        else:
            representative_description = desc
            break
    return representative_description

# Function to plot standard deviations
def plot_standard_deviations(column, group_labels, means, std_devs, p_value_1, p_value_2):
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.errorbar(means, group_labels, xerr=std_devs, fmt='o', capsize=5, capthick=2, elinewidth=2, linestyle='None')
    ax.set_title(f'Test for Equal Variances: {column} vs GROUP\nMultiple comparison intervals for the standard deviation, α = 0.05')
    ax.set_ylabel('GROUP')
    ax.set_xlabel('Standard Deviation')
    plt.figtext(0.95, 0.9, f"Multiple Comparisons\nP-Value (Levene's Test)\nP-Value: {p_value_1:.3f}", bbox={"facecolor": "lightgray", "alpha": 0.5, "pad": 5})
    plt.figtext(0.95, 0.8, f"P-Value (ANOVA or Welch)\nP-Value: {p_value_2:.3f}", bbox={"facecolor": "lightgray", "alpha": 0.5, "pad": 5})
    ax.set_yticks(range(len(group_labels)))
    ax.set_yticklabels(group_labels)
    plt.close(fig)
    return fig

# Function to handle user's choice for EVT or NPT
def handle_choice_1(choice):
    global user_choice_1
    user_choice_1 = choice
    print(f"User choice: {user_choice_1}")
    if user_choice_1 == 'Equal Variance Test':
        equal_variance_test()
    elif user_choice_1 == 'Non-Parametric Test':
        check_extreme_outliers()

# Function to handle user's choice for non-parametric test
def handle_choice_2(attr, group_df):
    def handle_choice_test(choice):
        global user_choice_2, global_results
        user_choice_2 = choice
        print(f"User choice for {attr}: {user_choice_2}")

        # Remove existing results for the chosen attribute and test type
        global_results = [result for result in global_results if not (result['Attributes'] == attr and result['Test Conducted'] == user_choice_2)]
        
        if user_choice_2 == "Mood's Median":
            results = moods_median_test(group_df, attr)
        elif user_choice_2 == 'Kruskal-Wallis':
            results = kruskal_wallis_test(group_df, attr)
        
        global_results.extend(results)
        results_df = pd.DataFrame(results)
        display(results_df)
        return results

    interact_widget = interact(handle_choice_test, choice=widgets.Dropdown(options=["Mood's Median", 'Kruskal-Wallis'], description='Choose Test:'))
    return interact_widget.widget.result

# Function to check skewness and proceed
def check_skewness_and_proceed():
    table1_df_global['Skewness'] = pd.to_numeric(table1_df_global['Skewness'], errors='coerce')
    skewness_check = table1_df_global['Skewness'].apply(lambda x: -2 <= x <= 2 if pd.notna(x) else True)
    if skewness_check.all():
        print("Skewness for all records is between -2 and 2. Proceed to user choice.")
        interact(handle_choice_1, choice=widgets.Dropdown(options=['Equal Variance Test', 'Non-Parametric Test'], description='Choose Test:'))
    else:
        print("Skewness for some records is outside the range of -2 and 2. Suggesting Non-Parametric Test (NPT) instead.")
        interact(handle_choice_1, choice=widgets.Dropdown(options=['Equal Variance Test', 'Non-Parametric Test'], description='Choose Test:'))

# Perform Anderson-Darling Test
def perform_anderson_darling_test():
    numerical_columns = [col for col in filtered_data.columns if pd.api.types.is_numeric_dtype(filtered_data[col])]
    print(f"Numerical columns: {numerical_columns}")
    data_for_testing = filtered_data[numerical_columns].values.flatten()
    statistic, critical_values, significance_level = anderson_darling_test(data_for_testing)
    p_value_threshold = 0.05
    if statistic < critical_values[2]:
        result = "Yes, fail to reject null"
        print(f"AD Test Result = {result}")
        print("p>0.05  fail to reject null")
        check_skewness_and_proceed()
    else:
        result = "No, reject null"
        print(f"AD Test Result = {result}")
        print("p<=0.05  reject null")
        perform_mgrt_test()

# Function to perform Minimum Group Records Test (MGRT)
def perform_mgrt_test():
    group_counts = filtered_data.groupby(['Attributes', 'GROUP']).size()
    print("Group counts:\n", group_counts)
    num_unique_groups = group_counts.groupby(level=0).size()
    print(f"Number of unique groups: {num_unique_groups}")
    result_MGRT = "Yes"
    for attr, group_count in group_counts.groupby(level=0):
        if 2 <= len(group_count) <= 9:
            print(f"Number of unique groups for {attr} is between 2 and 9.")
            if not all(group_count > 15):
                result_MGRT = "No"
                print(f"At least one group in {attr} has 15 or fewer records.")
            else:
                print(f"All groups in {attr} have more than 15 records.")
        elif 10 <= len(group_count) <= 12:
            print(f"Number of unique groups for {attr} is between 10 and 12.")
            if not all(group_count > 20):
                result_MGRT = "No"
                print(f"At least one group in {attr} has 20 or fewer records.")
            else:
                print(f"All groups in {attr} have more than 20 records.")
        else:
            print(f"Number of unique groups for {attr} is outside the specified range (2-12).")
    print(f"Result from Minimum Group Records Test = {result_MGRT}")
    if result_MGRT == "Yes":
        check_skewness_and_proceed()
    else:
        interact(handle_choice_1, choice=widgets.Dropdown(options=['Equal Variance Test', 'Non-Parametric Test'], description='Choose Test:'))

# Function to perform Equal Variance Test
def equal_variance_test():
    global global_levene_results, global_anova_welch_results
    descriptions = filtered_data['Attributes']
    representative_description = find_representative_description(descriptions)
    levene_results = []
    anova_welch_results = []
    numeric_columns = filtered_data.select_dtypes(include='number').columns
    grouped = filtered_data.groupby('Attributes')
    for attr, group_df in grouped:
        print(f"Processing group: Attributes={attr}")
        for column in numeric_columns:
            if column != 'GROUP':
                groups = group_df.groupby('GROUP')[column].apply(lambda x: x.dropna().values)
                group_labels = [label for label, group in zip(groups.index, groups) if len(group) > 0]
                means = [np.mean(group) for group in groups if len(group) > 0]
                std_devs = [np.std(group) for group in groups if len(group) > 0]
                if not means or not std_devs:
                    continue
                print(f"\nTesting column: {column}")
                for label, group in zip(group_labels, groups):
                    print(f"Group: {label}, Data: {group}")
                try:
                    stat, p_value_1 = levene(*[group for group in groups if len(group) > 0])
                    if np.isnan(p_value_1):
                        raise ValueError("NaN P-Value")
                except:
                    p_value_1 = np.nan
                    levene_results.append({
                        'Column': column,
                        'Levene P-Value': 'N/A',
                        'Result': 'N/A',
                        'Attributes': attr
                    })
                    anova_welch_results.append({
                        'Column': column,
                        'ANOVA/Welch P-Value': 'N/A',
                        'Result': 'N/A',
                        'Test Type': 'N/A',
                        'Attributes': attr
                    })
                    continue
                levene_result = 'Equal' if p_value_1 > 0.05 else 'Diff'
                levene_results.append({
                    'Column': column,
                    'Levene P-Value': p_value_1,
                    'Levene Statistic': stat,
                    'Result': levene_result,
                    'Attributes': attr
                })
                try:
                    if p_value_1 > 0.05:
                        print(f"Yes, Parametric ANOVA for {column}")
                        anova_result = f_oneway(*[group for group in groups if len(group) > 0])
                        p_value_2 = anova_result.pvalue
                        if np.isnan(p_value_2):
                            raise ValueError("NaN P-Value")
                        if p_value_2 > 0.05:
                            result = 'Equal'
                            test_type = 'ANOVA'
                        else:
                            result = 'Diff'
                            test_type = 'ANOVA'
                    else:
                        print(f"No, Parametric Welch ANOVA for {column}")
                        desc_stats = [DescrStatsW(group) for group in groups if len(group) > 0]
                        cm = CompareMeans(*desc_stats)
                        welch_result = cm.ttest_ind(usevar='unequal')
                        p_value_2 = welch_result[1]
                        if np.isnan(p_value_2):
                            raise ValueError("NaN P-Value")
                        if p_value_2 > 0.05:
                            result = 'Equal'
                            test_type = 'Welch ANOVA'
                        else:
                            result = 'Diff'
                            test_type = 'Welch ANOVA'
                except:
                    p_value_2 = np.nan
                    result = 'N/A'
                    test_type = 'N/A'
                fig = plot_standard_deviations(column, group_labels, means, std_devs, p_value_1, p_value_2)
                plots_dict.setdefault('standard_deviations', []).append((f'{attr}_{column}', fig))
                anova_welch_results.append({
                    'Column': column,
                    'ANOVA/Welch P-Value': p_value_2,
                    'Result': result,
                    'Test Type': test_type,
                    'Attributes': attr
                })
    global_levene_results = pd.DataFrame(levene_results)
    global_anova_welch_results = pd.DataFrame(anova_welch_results)
    # Format the DataFrames as requested
    levene_df_pivot = pd.pivot_table(global_levene_results, index='Attributes', columns='Column', values='Levene P-Value', aggfunc='first')
    anova_welch_df_pivot = pd.pivot_table(global_anova_welch_results, index='Attributes', columns='Column', values='ANOVA/Welch P-Value', aggfunc='first')
    # DataFrames without p-values
    levene_df_simple = global_levene_results.pivot(index='Attributes', columns='Column', values='Result')
    anova_welch_df_simple = global_anova_welch_results.pivot(index='Attributes', columns='Column', values='Result')
    print("Levene's Test Results (with P-Values):")
    display(levene_df_pivot)
    print("\nANOVA/Welch Test Results (with P-Values):")
    display(anova_welch_df_pivot)
    print("\nLevene's Test Results (Simplified):")
    display(levene_df_simple)
    print("\nANOVA/Welch Test Results (Simplified):")
    display(anova_welch_df_simple)

# Function to perform Mood's Median Test
def moods_median_test(group_df, attr):
    numeric_columns = group_df.select_dtypes(include='number').columns
    results = []
    for column in numeric_columns:
        if column != 'GROUP':
            groups = [group[column].dropna().values for name, group in group_df.groupby('GROUP')]
            groups = [group for group in groups if len(group) > 0]
            if len(groups) < 2:
                continue
            try:
                stat, p_value, med, tbl = median_test(*groups)
                if p_value > 0.05:
                    result = 'Equal'
                else:
                    result = 'Diff'
                group_stats = group_df.groupby('GROUP')[column].describe().unstack()
                result_data = {
                    'Attributes': attr,
                    'Column': column,
                    'Group': group_df['GROUP'].unique().tolist(),
                    'Median': group_stats['50%'].to_dict(),
                    'N': group_stats['count'].to_dict(),
                    'Q1-Q3': group_stats[['25%', '75%']].to_dict(),
                    'H-Value': stat,
                    'P-Value': p_value,
                    'Result': result,
                    'Test Conducted': "Mood's Median"
                }
                results.append(result_data)
            except ValueError as e:
                result_data = {
                    'Attributes': attr,
                    'Column': column,
                    'Group': group_df['GROUP'].unique().tolist(),
                    'Median': np.nan,
                    'N': len(group_df),
                    'Q1-Q3': 'N/A',
                    'H-Value': np.nan,
                    'P-Value': np.nan,
                    'Result': 'N/A',
                    'Test Conducted': "Mood's Median"
                }
                results.append(result_data)
    return results

# Function to perform Kruskal-Wallis Test
def kruskal_wallis_test(group_df, attr):
    numeric_columns = group_df.select_dtypes(include='number').columns
    results = []
    for column in numeric_columns:
        if column != 'GROUP':
            groups = [group[column].dropna().values for name, group in group_df.groupby('GROUP')]
            groups = [group for group in groups if len(group) > 0]
            if len(groups) < 2:
                continue
            try:
                stat, p_value = kruskal(*groups)
                if p_value > 0.05:
                    result = 'Equal'
                else:
                    result = 'Diff'
                group_stats = group_df.groupby('GROUP')[column].describe().unstack()
                result_data = {
                    'Attributes': attr,
                    'Column': column,
                    'Group': group_df['GROUP'].unique().tolist(),
                    'Median': group_stats['50%'].to_dict(),
                    'N': group_stats['count'].to_dict(),
                    'Q1-Q3': group_stats[['25%', '75%']].to_dict(),
                    'H-Value': stat,
                    'P-Value': p_value,
                    'Result': result,
                    'Test Conducted': "Kruskal-Wallis"
                }
                results.append(result_data)
            except ValueError as e:
                result_data = {
                    'Attributes': attr,
                    'Column': column,
                    'Group': group_df['GROUP'].unique().tolist(),
                    'Median': np.nan,
                    'N': len(group_df),
                    'Q1-Q3': 'N/A',
                    'H-Value': np.nan,
                    'P-Value': np.nan,
                    'Result': 'N/A',
                    'Test Conducted': "Kruskal-Wallis"
                }
                results.append(result_data)
    return results

# Function to perform tests by attribute
def perform_tests_by_attribute():
    all_results = []
    for attr in filtered_data['Attributes'].unique():
        group_df = filtered_data[filtered_data['Attributes'] == attr]
        print(f"\nChoose the test to conduct for {attr}:")
        results_df = handle_choice_2(attr, group_df)
        if results_df is not None:  # Ensure the results are valid
            all_results.append(results_df)

    # Collect the results from each attribute
    all_results_list = []
    for results_df in all_results:
        if isinstance(results_df, list):  # Check if it's a list
            all_results_list.extend(results_df)

    # Update the global results list
    global global_results
    global_results.extend(all_results_list)

# Function to check extreme outliers
def check_extreme_outliers():
    def find_extreme_outliers(series):
        q1 = series.quantile(0.25)
        q3 = series.quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        mean = series.mean()
        outliers = series[(series < lower_bound) | (series > upper_bound)]
        distances = (outliers - mean).abs()
        return outliers, distances

    outlier_present = False
    outliers_list = []
    numeric_columns = filtered_data.select_dtypes(include='number').columns
    grouped = filtered_data.groupby('Attributes')
    for attr, group_df in grouped:
        for column in numeric_columns:
            if column != 'GROUP':
                valid_groups = group_df['GROUP'].dropna().unique()
                if len(valid_groups) > 0:  # Ensure there are valid groups to plot
                    groups = group_df.groupby('GROUP')[column].apply(lambda x: x.dropna().values)
                    valid_groups = [group for group in groups if len(group) > 0]
                    if len(valid_groups) < 2:
                        continue
                    plt.figure(figsize=(10, 6))
                    sns.boxplot(x='GROUP', y=column, data=group_df)
                    plt.title(f"Boxplot of {column} by GROUP for Attributes={attr}")
                    plt.close()
                    plots_dict.setdefault('boxplot', []).append((f'{attr}_{column}', plt.gcf()))
                for name, group in group_df.groupby('GROUP'):
                    outliers, distances = find_extreme_outliers(group[column])
                    if not outliers.empty:
                        outlier_present = True
                        print(f"Column: {column}, Group: {name}, Attributes: {attr}")
                        print("Outliers and their distances from the mean:")
                        for value, distance in zip(outliers, distances):
                            print(f"Outlier: {value}, Distance from mean: {distance}")
                            outliers_list.append({
                                'Column': column,
                                'Group': name,
                                'Attributes': attr,
                                'Outlier': value,
                                'Distance from Mean': distance
                            })
    result = "Yes" if outlier_present else "No"
    print(f"Extreme outliers present: {result}")
    outliers_df = pd.DataFrame(outliers_list)
    display(outliers_df)
    perform_tests_by_attribute()

# Plotting functions for visualization
def plot_tukey_CIs(data, group_col, value_col):
    try:
        from statsmodels.stats.multicomp import pairwise_tukeyhsd
        tukey_result = pairwise_tukeyhsd(endog=data[value_col], groups=data[group_col], alpha=0.05)
        tukey_result.plot_simultaneous()
        plt.title(f'Tukey Simultaneous 95% CIs\nDifferences of Means for {value_col}')
        plt.xlabel('Difference of Means')
        plt.grid()
        plt.close()
        return plt.gcf()
    except Exception as e:
        print(f"Skipping Tukey HSD plot for {value_col}: {e}")
        return None

def plot_interval(data, group_col, value_col):
    sns.pointplot(x=group_col, y=value_col, data=data, capsize=.1, errwidth=1.5, ci='sd')
    plt.title(f'Interval Plot of {value_col} vs {group_col}\n95% CI for the Mean')
    plt.ylabel(value_col)
    plt.xlabel(group_col)
    plt.grid()
    plt.close()
    return plt.gcf()

def plot_individual_value(data, group_col, value_col):
    sns.stripplot(x=group_col, y=value_col, data=data, jitter=True)
    sns.pointplot(x=group_col, y=value_col, data=data, join=True, ci='sd', color='red')
    plt.title(f'Individual Value Plot of {value_col} vs {group_col}')
    plt.ylabel(value_col)
    plt.xlabel(group_col)
    plt.grid()
    plt.close()
    return plt.gcf()

def plot_boxplot(data, group_col, value_col):
    # Drop rows with NaN values in the group or value columns
    data = data.dropna(subset=[group_col, value_col])
    
    # Check if there are at least two groups with data
    if data[group_col].nunique() < 2:
        print(f"Not enough groups to plot boxplot for {value_col}. Skipping.")
        return None
    
    try:
        sns.boxplot(x=group_col, y=value_col, data=data)
        sns.pointplot(x=group_col, y=value_col, data=data, join=True, ci='sd', color='red')
        plt.title(f'Boxplot of {value_col}')
        plt.ylabel(value_col)
        plt.xlabel(group_col)
        plt.grid()
        plt.close()
        return plt.gcf()
    except ValueError as e:
        print(f"Error while plotting boxplot for {value_col}: {e}")
        return None

def plot_residuals(data, group_col, value_col):
    from statsmodels.formula.api import ols

    # Sanitize column names for use in the formula
    sanitized_value_col = value_col.replace('-', '_').replace(' ', '_')
    sanitized_group_col = group_col.replace('-', '_').replace(' ', '_')
    
    # Drop rows with NaN or infinite values in the relevant columns
    data = data.dropna(subset=[value_col, group_col])
    data = data[np.isfinite(data[value_col])]
    
    # Ensure there is more than one unique group and enough data to perform OLS
    if data[sanitized_group_col].nunique() < 2 or len(data) < 2:
        print(f"Not enough data to fit OLS model for {value_col}. Skipping.")
        return None
    
    # Rename columns for OLS model compatibility
    data = data.rename(columns={value_col: sanitized_value_col, group_col: sanitized_group_col})
    
    # Fit the OLS model
    formula = f'{sanitized_value_col} ~ {sanitized_group_col}'
    try:
        model = ols(formula, data=data).fit()
    except ValueError as e:
        print(f"Error fitting OLS model for {value_col}: {e}")
        return None
    
    residuals = model.resid
    fitted = model.fittedvalues
    
    fig, axs = plt.subplots(2, 2, figsize=(12, 10))
    
    sns.histplot(residuals, bins=15, kde=True, ax=axs[0, 0])
    axs[0, 0].set_title('Histogram of Residuals')
    
    stats.probplot(residuals, dist="norm", plot=axs[0, 1])
    axs[0, 1].set_title('Normal Probability Plot')
    
    sns.scatterplot(x=fitted, y=residuals, ax=axs[1, 0])
    axs[1, 0].axhline(0, color='r', linestyle='--')
    axs[1, 0].set_title('Residuals vs Fitted Values')
    
    sns.lineplot(x=np.arange(len(residuals)), y=residuals, ax=axs[1, 1])
    axs[1, 1].set_title('Residuals vs Order')
    axs[1, 1].axhline(0, color='r', linestyle='--')
    
    plt.tight_layout()
    plt.close(fig)
    return fig

# Adjusted function to plot all visualizations for each numerical column against 'GROUP' for each Attribute
def plot_all_visualizations(data):
    group_col = 'GROUP'
    numeric_columns = data.select_dtypes(include='number').columns
    
    # Loop through each attribute in the data
    for attr in data['Attributes'].unique():
        attr_data = data[data['Attributes'] == attr]
        
        for value_col in numeric_columns:
            if value_col != group_col:
                print(f"Plotting for Attribute: {attr}, Column: {value_col}")
                
                # Create Tukey's HSD plot
                fig = plot_tukey_CIs(attr_data, group_col, value_col)
                if fig:
                    plots_dict.setdefault('tukey_CIs', []).append((f'{attr}_{value_col}', fig))
                
                # Create Interval plot
                fig = plot_interval(attr_data, group_col, value_col)
                if fig:
                    plots_dict.setdefault('interval', []).append((f'{attr}_{value_col}', fig))
                
                # Create Individual Value plot
                fig = plot_individual_value(attr_data, group_col, value_col)
                if fig:
                    plots_dict.setdefault('individual_value', []).append((f'{attr}_{value_col}', fig))
                
                # Create Boxplot
                fig = plot_boxplot(attr_data, group_col, value_col)
                if fig:
                    plots_dict.setdefault('boxplot', []).append((f'{attr}_{value_col}', fig))
                
                # Create Residuals plot
                fig = plot_residuals(attr_data, group_col, value_col)
                if fig:
                    plots_dict.setdefault('residuals', []).append((f'{attr}_{value_col}', fig))

# Save plots to Excel file
def save_plots_to_excel(plots_dict, filename):
    with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
        for plot_type, plots in plots_dict.items():
            worksheet_name = plot_type[:31]  # Worksheet names must be <= 31 chars
            worksheet = writer.book.add_worksheet(worksheet_name)
            row = 0
            for value_col, fig in plots:
                # Save the figure to a BytesIO object
                img_data = io.BytesIO()
                fig.savefig(img_data, format='png')
                img_data.seek(0)
                # Open the image with PIL
                image = Image.open(img_data)
                # Convert to a format xlsxwriter can handle
                image_data = io.BytesIO()
                image.save(image_data, format='png')
                image_data.seek(0)
                # Insert the image into the Excel sheet
                worksheet.insert_image(row, 0, '', {'image_data': image_data})
                worksheet.write(row, 1, value_col)
                row += 30  # Adjust the row position for the next plot

# Interact to trigger the Anderson-Darling test and proceed based on the results
perform_anderson_darling_test()

# Example usage with filtered_data
plot_all_visualizations(filtered_data)



Numerical columns: ['Core', 'Comps_Thin', 'Comps_Thick', 'Comps', 'F_Thick', 'Kfactor_Init-1', 'EdgeCol_Bot', 'EdgeCol_Top', 'L_Shrink', 'W_Shrink']
AD Test Result = No, reject null
p<=0.05  reject null
Group counts:
 Attributes                GROUP  
0.5 High Density Cgf-Cgf  EVONIK     64
                          SILSTAB    38
1.5 Flat Blk-Blk          EVONIK     36
                          SILSTAB    25
2.6 Flat Blk-Blk          EVONIK     15
                          SILSTAB    11
Q Taper Blk-Blk           EVONIK     23
                          SILSTAB    18
dtype: int64
Number of unique groups: Attributes
0.5 High Density Cgf-Cgf    2
1.5 Flat Blk-Blk            2
2.6 Flat Blk-Blk            2
Q Taper Blk-Blk             2
dtype: int64
Number of unique groups for 0.5 High Density Cgf-Cgf is between 2 and 9.
All groups in 0.5 High Density Cgf-Cgf have more than 15 records.
Number of unique groups for 1.5 Flat Blk-Blk is between 2 and 9.
All groups in 1.5 Flat Blk-Blk have more t

interactive(children=(Dropdown(description='Choose Test:', options=('Equal Variance Test', 'Non-Parametric Tes…

Plotting for Attribute: 0.5 High Density Cgf-Cgf, Column: Core
Plotting for Attribute: 0.5 High Density Cgf-Cgf, Column: Comps_Thin
Plotting for Attribute: 0.5 High Density Cgf-Cgf, Column: Comps_Thick
Plotting for Attribute: 0.5 High Density Cgf-Cgf, Column: Comps
Plotting for Attribute: 0.5 High Density Cgf-Cgf, Column: F_Thick
Plotting for Attribute: 0.5 High Density Cgf-Cgf, Column: Kfactor_Init-1
Plotting for Attribute: 0.5 High Density Cgf-Cgf, Column: EdgeCol_Bot
Skipping Tukey HSD plot for EdgeCol_Bot: Axis limits cannot be NaN or Inf
Not enough groups to plot boxplot for EdgeCol_Bot. Skipping.
Not enough data to fit OLS model for EdgeCol_Bot. Skipping.
Plotting for Attribute: 0.5 High Density Cgf-Cgf, Column: EdgeCol_Top
Skipping Tukey HSD plot for EdgeCol_Top: Axis limits cannot be NaN or Inf
Not enough groups to plot boxplot for EdgeCol_Top. Skipping.
Not enough data to fit OLS model for EdgeCol_Top. Skipping.
Plotting for Attribute: 0.5 High Density Cgf-Cgf, Column: L_Shrin

<Figure size 640x480 with 0 Axes>

In [6]:
# Check if the user choice is 'Non-Parametric Test'
if user_choice_1 == 'Non-Parametric Test':
    # Create a DataFrame from global_results
    npt_results_df = pd.DataFrame(global_results)
    
    # Ensure the DataFrame only keeps the last occurrence of each 'Attributes'/'Column' combination
    npt_results_df_filtered = npt_results_df.drop_duplicates(subset=['Attributes', 'Column'], keep='last')
    
    # Display the filtered DataFrame to the user
    print("Non Paramteric Test Results:")
    display(npt_results_df_filtered)
else:
    # Pivot for Levene P-Value
    levene_pivot_pvalue = global_levene_results.pivot(index='Attributes', columns='Column', values='Levene P-Value')

    # Pivot for Levene Statistic
    levene_pivot_statistic = global_levene_results.pivot(index='Attributes', columns='Column', values='Levene Statistic')

    # Pivot for Result
    levene_pivot_result = global_levene_results.pivot(index='Attributes', columns='Column', values='Result')

    # Display the pivot tables
    print("Levene's Test P-Values (Pivoted):")
    display(levene_pivot_pvalue)

    print("Levene's Test Statistics (Pivoted):")
    display(levene_pivot_statistic)

    print("Levene's Test Results (Pivoted):")
    display(levene_pivot_result)

    # Pivot for ANOVA/Welch P-Value
    anova_welch_pivot_pvalue = global_anova_welch_results.pivot(index='Attributes', columns='Column', values='ANOVA/Welch P-Value')

    # Pivot for Result
    anova_welch_pivot_result = global_anova_welch_results.pivot(index='Attributes', columns='Column', values='Result')


    # Display the pivot tables
    print("ANOVA/Welch Test P-Values (Pivoted):")
    display(anova_welch_pivot_pvalue)

    print("ANOVA/Welch Test Results (Pivoted):")
    display(anova_welch_pivot_result)





Levene's Test P-Values (Pivoted):


Column,Comps,Comps_Thick,Comps_Thin,Core,EdgeCol_Bot,EdgeCol_Top,F_Thick,Kfactor_Init-1,L_Shrink,W_Shrink
Attributes,,,,,,,,,,
0.5 High Density Cgf-Cgf,0.142411,0.132714,0.716330,0.116278,NaN,NaN,0.024073,N/A,N/A,N/A
1.5 Flat Blk-Blk,0.720560,0.437225,0.783853,0.722026,NaN,N/A,0.287955,N/A,N/A,N/A
2.6 Flat Blk-Blk,0.442408,0.794669,0.068087,0.510908,0.038112,0.628695,0.187435,N/A,0.581987,0.799517
Q Taper Blk-Blk,0.185458,0.016714,0.368400,0.632013,NaN,N/A,0.943334,N/A,N/A,N/A


Levene's Test Statistics (Pivoted):


Column,Comps,Comps_Thick,Comps_Thin,Core,EdgeCol_Bot,EdgeCol_Top,F_Thick,Kfactor_Init-1,L_Shrink,W_Shrink
Attributes,,,,,,,,,,
0.5 High Density Cgf-Cgf,2.186030,2.297774,0.132786,2.510072,NaN,NaN,5.247794,NaN,NaN,NaN
1.5 Flat Blk-Blk,0.129187,0.611836,0.075931,0.127796,NaN,NaN,1.149814,NaN,NaN,NaN
2.6 Flat Blk-Blk,0.610043,0.069256,3.650017,0.445370,4.867328,0.240507,1.841214,NaN,0.312183,0.066083
Q Taper Blk-Blk,1.816921,6.252426,0.828144,0.232991,NaN,NaN,0.005118,NaN,NaN,NaN


Levene's Test Results (Pivoted):


Column,Comps,Comps_Thick,Comps_Thin,Core,EdgeCol_Bot,EdgeCol_Top,F_Thick,Kfactor_Init-1,L_Shrink,W_Shrink
Attributes,,,,,,,,,,
0.5 High Density Cgf-Cgf,Equal,Equal,Equal,Equal,NaN,NaN,Diff,N/A,N/A,N/A
1.5 Flat Blk-Blk,Equal,Equal,Equal,Equal,NaN,N/A,Equal,N/A,N/A,N/A
2.6 Flat Blk-Blk,Equal,Equal,Equal,Equal,Diff,Equal,Equal,N/A,Equal,Equal
Q Taper Blk-Blk,Equal,Diff,Equal,Equal,NaN,N/A,Equal,N/A,N/A,N/A


ANOVA/Welch Test P-Values (Pivoted):


Column,Comps,Comps_Thick,Comps_Thin,Core,EdgeCol_Bot,EdgeCol_Top,F_Thick,Kfactor_Init-1,L_Shrink,W_Shrink
Attributes,,,,,,,,,,
0.5 High Density Cgf-Cgf,0.000000,0.039837,0.000008,0.193439,NaN,NaN,0.330451,N/A,N/A,N/A
1.5 Flat Blk-Blk,0.000000,0.000000,0.000000,0.051126,NaN,N/A,0.197917,N/A,N/A,N/A
2.6 Flat Blk-Blk,0.373999,0.077453,0.883901,0.917061,0.018865,0.110370,0.582957,N/A,0.321736,0.857628
Q Taper Blk-Blk,0.117635,0.696022,0.014775,0.708858,NaN,N/A,0.968449,N/A,N/A,N/A


ANOVA/Welch Test Results (Pivoted):


Column,Comps,Comps_Thick,Comps_Thin,Core,EdgeCol_Bot,EdgeCol_Top,F_Thick,Kfactor_Init-1,L_Shrink,W_Shrink
Attributes,,,,,,,,,,
0.5 High Density Cgf-Cgf,Diff,Diff,Diff,Equal,NaN,NaN,Equal,N/A,N/A,N/A
1.5 Flat Blk-Blk,Diff,Diff,Diff,Equal,NaN,N/A,Equal,N/A,N/A,N/A
2.6 Flat Blk-Blk,Equal,Equal,Equal,Equal,Diff,Equal,Equal,N/A,Equal,Equal
Q Taper Blk-Blk,Equal,Equal,Diff,Equal,NaN,N/A,Equal,N/A,N/A,N/A


In [7]:
# Display the unique GROUP values for selection
unique_groups = table1_df_global['GROUP'].unique()
print("Unique GROUP values:", unique_groups)

# User input for selecting Trial and Incumbent groups
trial_group = input("Please select the Trial group from the above unique GROUP values: ")
incumbent_group = input("Please select the Incumbent group from the above unique GROUP values: ")

test_choice=user_choice_1

if test_choice == 'Equal Variance Test':
    # Transform Levene's Test results table
    levene_melted = levene_pivot_pvalue.reset_index().melt(id_vars=['Attributes'], var_name='Column', value_name='P-Value')
    levene_melted = levene_melted.dropna(subset=['P-Value'])

    # Transform ANOVA Welch's Test results table
    anova_melted = anova_welch_pivot_pvalue.reset_index().melt(id_vars=['Attributes'], var_name='Column', value_name='P-Value')
    anova_melted = anova_melted.dropna(subset=['P-Value'])

# Get the corresponding p-values based on the user's choice
if test_choice == 'Non-Parametric Test':
    p_values = npt_results_df_filtered[['Attributes','Column','P-Value']]
elif test_choice == 'Equal Variance Test':
    equal_variance_choice = input("Please choose between Levene's Test or Welch's ANOVA: ").strip().lower()
    if equal_variance_choice == "levene's test":
        p_values = levene_melted[['Attributes','Column','P-Value']]
    elif equal_variance_choice == "welch's anova":
        p_values = anova_melted[['Attributes','Column','P-Value']]
    else:
        raise ValueError("Invalid choice for Equal Variance Test.")
else:
    raise ValueError("Invalid test choice.")

# Create the desired table
result_df = pd.DataFrame()

# Filter the necessary data
trial_data = table1_df_global[table1_df_global['GROUP'] == trial_group]
incumbent_data = table1_df_global[table1_df_global['GROUP'] == incumbent_group]


# Merge trial and incumbent data based on the 'Attributes' column
merged_data = pd.merge(trial_data, incumbent_data, on=['Attributes','Variable'], suffixes=('_Trial', '_Incumbent'))

# Ensure the numerical columns are of numeric type
merged_data['Mean_Trial'] = pd.to_numeric(merged_data['Mean_Trial'], errors='coerce')
merged_data['Mean_Incumbent'] = pd.to_numeric(merged_data['Mean_Incumbent'], errors='coerce')
merged_data['N_Trial'] = pd.to_numeric(merged_data['N_Trial'], errors='coerce')
merged_data['N_Incumbent'] = pd.to_numeric(merged_data['N_Incumbent'], errors='coerce')

# Calculate the required columns
result_df['Product'] = merged_data['Attributes']
result_df['Group'] = merged_data['Variable']
result_df['Sample Size (Trial)'] = merged_data['N_Trial']
result_df['Sample Size (Incumbent)'] = merged_data['N_Incumbent']
result_df['Trial'] = merged_data['Mean_Trial']
result_df['Incumbent'] = merged_data['Mean_Incumbent']
result_df['Diff (Trial - Incumbent)'] = merged_data['Mean_Trial'] - merged_data['Mean_Incumbent']

# Merge p-values with the result_df
result_df = result_df.merge(p_values, left_on=['Product','Group'], right_on=['Attributes','Column'], how='left')

# Convert P-Value column to numeric, coercing errors to NaN
result_df['P-Value'] = pd.to_numeric(result_df['P-Value'], errors='coerce')

# Calculate Statistically Different column
result_df['Statistically Different'] = result_df['P-Value'] < 0.05
result_df['Statistically Different'] = result_df['Statistically Different'].apply(lambda x: 'YES' if x else 'NO')

# Display the resulting DataFrame
result_df=result_df.drop(columns=['Column','Attributes'])
result_df

Unique GROUP values: ['EVONIK' 'SILSTAB']


Please select the Trial group from the above unique GROUP values:  SILSTAB
Please select the Incumbent group from the above unique GROUP values:  EVONIK
Please choose between Levene's Test or Welch's ANOVA:  Levene's Test


,Product,Group,Sample Size (Trial),Sample Size (Incumbent),Trial,Incumbent,Diff (Trial - Incumbent),P-Value,Statistically Different
0,0.5 High Density Cgf-Cgf,Comps,38,64,103.087000,107.776000,-4.689000,0.142411,NO
1,0.5 High Density Cgf-Cgf,Comps_Thick,38,64,101.942000,104.908000,-2.966000,0.132714,NO
2,0.5 High Density Cgf-Cgf,Comps_Thin,38,64,99.907000,107.266000,-7.359000,0.716330,NO
3,0.5 High Density Cgf-Cgf,Core,38,64,4.049000,4.080000,-0.031000,0.116278,NO
4,0.5 High Density Cgf-Cgf,EdgeCol_Bot,38,64,NaN,NaN,NaN,NaN,NO
5,0.5 High Density Cgf-Cgf,EdgeCol_Top,38,64,NaN,NaN,NaN,NaN,NO
6,0.5 High Density Cgf-Cgf,F_Thick,38,64,0.445000,0.444000,0.001000,0.024073,YES
7,0.5 High Density Cgf-Cgf,Kfactor_Init-1,38,64,0.000000,0.000000,0.000000,NaN,NO
8,0.5 High Density Cgf-Cgf,L_Shrink,38,64,0.000000,0.000000,0.000000,NaN,NO
9,0.5 High Density Cgf-Cgf,W_Shrink,38,64,0.000000,0.000000,0.000000,NaN,NO


In [8]:
# Pivot the DataFrame
pivot_df = result_df.melt(id_vars=['Product', 'Group'], 
                   value_vars=['Sample Size (Trial)', 'Sample Size (Incumbent)', 'Trial', 'Incumbent', 'Diff (Trial - Incumbent)', 'P-Value', 'Statistically Different'],
                   var_name='Metric', value_name='Value')

pivot_df = pivot_df.pivot_table(index=['Product', 'Metric'], columns='Group', values='Value', aggfunc='first').reset_index()

# Flatten the columns
pivot_df.columns = ['_'.join(col).strip() if type(col) is tuple else col for col in pivot_df.columns.values]
pivot_df = pivot_df.rename(columns={'Product_': 'Product', 'Metric_': 'Metric'})

# Define the desired order of metrics
metric_order = [
    'Sample Size (Trial)', 'Sample Size (Incumbent)', 'Trial', 'Incumbent',
    'Diff (Trial - Incumbent)', 'P-Value', 'Statistically Different'
]

# Sort the DataFrame by Product and Metric according to the defined order
pivot_df['Metric'] = pd.Categorical(pivot_df['Metric'], categories=metric_order, ordered=True)
pivot_df = pivot_df.sort_values(by=['Product', 'Metric']).reset_index(drop=True)
table3_df_global=pivot_df.copy()

In [9]:
table3_df_global

,Product,Metric,Comps,Comps_Thick,Comps_Thin,Core,EdgeCol_Bot,EdgeCol_Top,F_Thick,Kfactor_Init-1,L_Shrink,W_Shrink
0,0.5 High Density Cgf-Cgf,Sample Size (Trial),38,38,38,38,38,38,38,38,38,38
1,0.5 High Density Cgf-Cgf,Sample Size (Incumbent),64,64,64,64,64,64,64,64,64,64
2,0.5 High Density Cgf-Cgf,Trial,103.087000,101.942000,99.907000,4.049000,NaN,NaN,0.445000,0.000000,0.000000,0.000000
3,0.5 High Density Cgf-Cgf,Incumbent,107.776000,104.908000,107.266000,4.080000,NaN,NaN,0.444000,0.000000,0.000000,0.000000
4,0.5 High Density Cgf-Cgf,Diff (Trial - Incumbent),-4.689000,-2.966000,-7.359000,-0.031000,NaN,NaN,0.001000,0.000000,0.000000,0.000000
5,0.5 High Density Cgf-Cgf,P-Value,0.142411,0.132714,0.716330,0.116278,NaN,NaN,0.024073,NaN,NaN,NaN
6,0.5 High Density Cgf-Cgf,Statistically Different,NO,NO,NO,NO,NO,NO,YES,NO,NO,NO
7,1.5 Flat Blk-Blk,Sample Size (Trial),25,25,25,25,25,25,25,25,25,25
8,1.5 Flat Blk-Blk,Sample Size (Incumbent),36,36,36,36,36,36,36,36,36,36
9,1.5 Flat Blk-Blk,Trial,21.052000,20.885000,21.195000,1.639000,NaN,0.000000,1.470000,0.000000,0.000000,0.000000


In [11]:
import pandas as pd
import numpy as np

# Assuming table3_df_global is already defined
# Filter the DataFrame to include only rows where the Metric is 'P-Value' or 'Diff (Trial - Incumbent)'
filtered_df = table3_df_global[table3_df_global['Metric'].isin(['P-Value', 'Diff (Trial - Incumbent)'])]

# Convert columns to numeric except 'Product' and 'Metric'
columns_to_convert = filtered_df.columns.difference(['Product', 'Metric'])
filtered_df[columns_to_convert] = filtered_df[columns_to_convert].apply(pd.to_numeric, errors='coerce')

# Replace 0 with NaN to ignore them in calculations
filtered_df = filtered_df.replace(0, np.nan)

# Create a new DataFrame to store the results
results = []

# Iterate over each product
for product in filtered_df['Product'].unique():
    # Filter for the current product
    product_df = filtered_df[filtered_df['Product'] == product]

    # Get the P-values and differences
    p_values = product_df[product_df['Metric'] == 'P-Value']
    diffs = product_df[product_df['Metric'] == 'Diff (Trial - Incumbent)']
    
    # Iterate over each metric
    for col in columns_to_convert:
        p_value = p_values[col].values[0] if col in p_values.columns else np.nan
        diff_value = diffs[col].values[0] if col in diffs.columns else np.nan
        
        # Determine if statistically different
        stat_diff = 'Yes' if not np.isnan(p_value) and p_value < 0.05 else 'No'
        
        # Determine assessment
        assessment = 'Higher' if diff_value > 0 else 'Lower' if diff_value < 0 else 'N/A'
        
        # Append the result if Statistically Different is Yes
        if stat_diff == 'Yes':
            results.append({
                'Product': product,
                'Metric': col,
                'Statistically Diff': stat_diff,
                'Avg Measured Diff': diff_value,
                'Assessment': assessment
            })

# Convert results to a DataFrame
table4_df_global = pd.DataFrame(results)

# Display the resulting DataFrame
table4_df_global


,Product,Metric,Statistically Diff,Avg Measured Diff,Assessment
0,0.5 High Density Cgf-Cgf,F_Thick,Yes,0.001000,Higher
1,2.6 Flat Blk-Blk,EdgeCol_Bot,Yes,0.077000,Higher
2,Q Taper Blk-Blk,Comps_Thick,Yes,0.249000,Higher


In [9]:
'''
import pandas as pd
import numpy as np

# Assuming table3_df_global is already defined
# Filter the DataFrame to include only rows where the Metric is 'P-Value'
p_value_df = table3_df_global[table3_df_global['Metric'] == 'P-Value']

# Convert columns to numeric except 'Product' and 'Metric'
columns_to_convert = p_value_df.columns.difference(['Product', 'Metric'])
p_value_df[columns_to_convert] = p_value_df[columns_to_convert].apply(pd.to_numeric, errors='coerce')

# Identify the numeric columns (excluding 'Product' and 'Metric')
numeric_columns = p_value_df.select_dtypes(include=['number']).columns

# Replace 0 with NaN to ignore them in mean calculations
p_value_df = p_value_df.replace(0, np.nan)

# Calculate the average P-value for each Metric across all numeric columns, ignoring 0 and NaN
average_p_values = p_value_df.groupby(['Metric'])[numeric_columns].mean().reset_index()

# Filter and process the DataFrame for 'P-Value' and 'Diff (Trial - Incumbent)'
filtered_df = table3_df_global[table3_df_global['Metric'].isin(['P-Value', 'Diff (Trial - Incumbent)'])]

# Convert columns to numeric except 'Metric'
filtered_df[columns_to_convert] = filtered_df[columns_to_convert].apply(pd.to_numeric, errors='coerce')

# Replace 0 with NaN to ignore them in mean calculations
filtered_df = filtered_df.replace(0, np.nan)

# Calculate the average for each Metric across all numeric columns
average_values = filtered_df.groupby(['Metric'])[numeric_columns].mean().reset_index()

# Transpose the DataFrame and add 'Statistically Different' and 'Assessment' columns
average_values = average_values.set_index('Metric').T.reset_index().rename(columns={'index': 'Metric'})

average_values['Statistically Different'] = average_values['P-Value'].apply(
    lambda x: 'Yes' if x < 0.05 else 'N/A' if np.isnan(x) else 'No'
)
average_values['Assessment'] = average_values['Diff (Trial - Incumbent)'].apply(
    lambda x: 'Higher' if x > 0 else 'N/A' if np.isnan(x) else 'Lower'
)

# Rename columns for clarity
average_values = average_values.rename(columns={'Metric': 'Metrics'})

# Select and reorder the relevant columns
table4_df_global = average_values[['Metrics', 'Diff (Trial - Incumbent)', 'P-Value', 'Statistically Different', 'Assessment']]

table4_df_global
'''

Metric,Metrics,Diff (Trial - Incumbent),P-Value,Statistically Different,Assessment
0,Comps,-3.684000,0.431485,No,Lower
1,Comps_Thick,-2.802500,0.284970,No,Lower
2,Comps_Thin,-5.050000,0.750091,No,Lower
3,Core,-0.021000,0.419152,No,Lower
4,F_Thick,0.002500,0.156014,No,Higher


In [12]:
# Check if the user choice is 'Non-Parametric Test'
if user_choice_1 == 'Non-Parametric Test':
    # Create a DataFrame from global_results
    table2_df_global=npt_results_df_filtered
else:
    table2_df_global=levene_pivot_result

table2_df_global=table2_df_global.reset_index()

In [13]:
table2_df_global=table2_df_global.rename(columns={'Attributes':'Product'})
table2_df_global

Column,Product,Comps,Comps_Thick,Comps_Thin,Core,EdgeCol_Bot,EdgeCol_Top,F_Thick,Kfactor_Init-1,L_Shrink,W_Shrink
0,0.5 High Density Cgf-Cgf,Equal,Equal,Equal,Equal,NaN,NaN,Diff,N/A,N/A,N/A
1,1.5 Flat Blk-Blk,Equal,Equal,Equal,Equal,NaN,N/A,Equal,N/A,N/A,N/A
2,2.6 Flat Blk-Blk,Equal,Equal,Equal,Equal,Diff,Equal,Equal,N/A,Equal,Equal
3,Q Taper Blk-Blk,Equal,Diff,Equal,Equal,NaN,N/A,Equal,N/A,N/A,N/A


In [14]:
table3_df_global

,Product,Metric,Comps,Comps_Thick,Comps_Thin,Core,EdgeCol_Bot,EdgeCol_Top,F_Thick,Kfactor_Init-1,L_Shrink,W_Shrink
0,0.5 High Density Cgf-Cgf,Sample Size (Trial),38,38,38,38,38,38,38,38,38,38
1,0.5 High Density Cgf-Cgf,Sample Size (Incumbent),64,64,64,64,64,64,64,64,64,64
2,0.5 High Density Cgf-Cgf,Trial,103.087000,101.942000,99.907000,4.049000,NaN,NaN,0.445000,0.000000,0.000000,0.000000
3,0.5 High Density Cgf-Cgf,Incumbent,107.776000,104.908000,107.266000,4.080000,NaN,NaN,0.444000,0.000000,0.000000,0.000000
4,0.5 High Density Cgf-Cgf,Diff (Trial - Incumbent),-4.689000,-2.966000,-7.359000,-0.031000,NaN,NaN,0.001000,0.000000,0.000000,0.000000
5,0.5 High Density Cgf-Cgf,P-Value,0.142411,0.132714,0.716330,0.116278,NaN,NaN,0.024073,NaN,NaN,NaN
6,0.5 High Density Cgf-Cgf,Statistically Different,NO,NO,NO,NO,NO,NO,YES,NO,NO,NO
7,1.5 Flat Blk-Blk,Sample Size (Trial),25,25,25,25,25,25,25,25,25,25
8,1.5 Flat Blk-Blk,Sample Size (Incumbent),36,36,36,36,36,36,36,36,36,36
9,1.5 Flat Blk-Blk,Trial,21.052000,20.885000,21.195000,1.639000,NaN,0.000000,1.470000,0.000000,0.000000,0.000000


In [15]:
# Function to round numerical columns to 3 decimal places
def round_numeric_columns(df, decimals=3):
    return df.applymap(lambda x: round(x, decimals) if isinstance(x, (int, float)) else x)

# Apply the rounding function to each DataFrame
table1_df_global = round_numeric_columns(table1_df_global)
table2_df_global = round_numeric_columns(table2_df_global)
table3_df_global = round_numeric_columns(table3_df_global)
table4_df_global = round_numeric_columns(table4_df_global)

In [16]:
table1_df_global

,Attributes,GROUP,Variable,N,N*,Mean,StDev,Minimum,Median,Maximum,Skewness
0,0.5 High Density Cgf-Cgf,EVONIK,Comps,64,0,107.776,4.468,98.580,107.380,123.210,0.679
1,0.5 High Density Cgf-Cgf,SILSTAB,Comps,38,0,103.087,3.754,90.350,103.315,110.640,-0.915
2,0.5 High Density Cgf-Cgf,EVONIK,Comps_Thick,64,0,104.908,6.349,90.600,103.770,122.070,0.414
3,0.5 High Density Cgf-Cgf,SILSTAB,Comps_Thick,38,0,101.942,7.876,81.960,102.790,115.890,-0.450
4,0.5 High Density Cgf-Cgf,EVONIK,Comps_Thin,64,0,107.266,7.881,88.610,107.435,125.840,-0.098
...,...,...,...,...,...,...,...,...,...,...,...
75,Q Taper Blk-Blk,SILSTAB,Kfactor_Init-1,18,0,0.000,0.000,0.000,0.000,0.000,0.000
76,Q Taper Blk-Blk,EVONIK,L_Shrink,23,0,0.000,0.000,0.000,0.000,0.000,0.000
77,Q Taper Blk-Blk,SILSTAB,L_Shrink,18,0,0.000,0.000,0.000,0.000,0.000,0.000
78,Q Taper Blk-Blk,EVONIK,W_Shrink,23,0,0.000,0.000,0.000,0.000,0.000,0.000


In [21]:
import pandas as pd
import io
from PIL import Image
from openpyxl import load_workbook, Workbook
from openpyxl.drawing.image import Image as OpenpyxlImage
from openpyxl.styles import Font, PatternFill, Border, Side

# Assuming table1_df_global, table2_df_global, table3_df_global, and table4_df_global are already defined

# Define the output path for the Excel file
output_path = 'test_trial_final_6.xlsx'

# Function to apply styles to a range of cells
def style_range(ws, cell_range, font=None, fill=None, border=None):
    rows = ws[cell_range]
    for row in rows:
        for cell in row:
            if font:
                cell.font = font
            if fill:
                cell.fill = fill
            if border:
                cell.border = border

# Function to write and style a DataFrame in the Excel sheet
def write_dataframe_to_excel(sheet, df, start_row, header_font, header_fill, thin_border):
    # Write the headers
    for col_num, col_name in enumerate(df.columns, start=1):
        cell = sheet.cell(row=start_row, column=col_num, value=col_name)
        cell.font = header_font
        cell.fill = header_fill
        cell.border = thin_border

    # Apply the header style
    style_range(sheet, f'A{start_row}:{chr(64 + len(df.columns))}{start_row}', font=header_font, fill=header_fill, border=thin_border)
    
    start_row += 1

    # Write the data
    for row_num, r in enumerate(df.itertuples(index=False), start=start_row):
        for col_num, value in enumerate(r, start=1):
            # Convert lists and dictionaries to strings
            if isinstance(value, list):
                value = ', '.join(map(str, value))
            elif isinstance(value, dict):
                value = ', '.join(f'{k}: {v}' for k, v in value.items())
            cell = sheet.cell(row=row_num, column=col_num, value=value)
            cell.border = thin_border

    # Update the start_row for the next DataFrame
    return row_num + 5  # Leave 5 empty rows between tables

# Create a new Excel file and write the first DataFrame to it using openpyxl
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    table1_df_global.to_excel(writer, sheet_name='tables', startrow=0, index=False)

# Load the workbook and the worksheet with openpyxl for further modifications
book = load_workbook(output_path)
sheet = book['tables']

# Define styles
header_font = Font(bold=True)
header_fill = PatternFill(start_color="D3D3D3", end_color="D3D3D3", fill_type="solid")
thin_border = Border(left=Side(style='thin'),
                     right=Side(style='thin'),
                     top=Side(style='thin'),
                     bottom=Side(style='thin'))
